## Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

# Tensorflow libraries
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#matplotlib style
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['legend.frameon'] = False

# project package
from src.plot_functions import plot_acc_loss
from src.training_functions import exp_decay, train_lrscheduler
from src.init_data import create_train_val_data_from_directory


In [2]:
# Create dataset
data_dir = "/home/titou/Documents/git/flower_image_class/data/flower_photos/"
data_dir = pathlib.Path(data_dir)

train_ds, validation_ds = create_train_val_data_from_directory(data_dir)

class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


2022-07-21 12:17:03.750078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 12:17:03.754369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 12:17:03.754533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 12:17:03.754881: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
resnet50 = tf.keras.models.load_model('../data/pretrained_models/resnet50_nopooling.h5')
resnet50.trainable=False

In [4]:
# Create the new model

    
def create_resnet(num_classes):

    model = Sequential([

        layers.Resizing(img_height, img_width),
        layers.Rescaling(1./255),

        resnet50,
        layers.MaxPool2D(),
        layers.Flatten(),
        layers.Dropout(0.2, seed=123),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes)
    ])


    opt=tf.keras.optimizers.Adam(learning_rate=1e-3)

    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

  

In [5]:
resnet = create_resnet(num_classes)
epochs = 10
resnet_history = train_lrscheduler(resnet, epochs, train_ds, validation_ds, exp_decay)
plot_acc_loss(resnet_history, epochs)



Epoch 1/10


2022-07-21 12:11:58.837975: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-07-21 12:11:59.322509: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-21 12:11:59.322859: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-21 12:11:59.322884: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-07-21 12:11:59.323280: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-21 12:11:59.323322: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


 5/92 [>.............................] - ETA: 3s - loss: 13.2127 - accuracy: 0.2750

2022-07-21 12:12:00.293820: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


92/92 [==============================] - 10s 72ms/step - loss: 1.6850 - accuracy: 0.7241 - val_loss: 0.4942 - val_accuracy: 0.8243 - lr: 0.0010
Epoch 2/10
92/92 [==============================] - 5s 55ms/step - loss: 0.2687 - accuracy: 0.9016 - val_loss: 0.5480 - val_accuracy: 0.8324 - lr: 0.0010
Epoch 3/10
69/92 [=====================>........] - ETA: 1s - loss: 0.1319 - accuracy: 0.9556

KeyboardInterrupt: 